In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tqdm
from scipy import sparse

# Generate some indices
Even the sparse matrices won't fit in memory. So we will have to loop through them when making predictions or sampling random items.

In [2]:
#count number of items:
indptr = [0]

for chunkID in range(10):
    scores = np.load(f'../processed_data/AmpC_all{chunkID}.npy')
    indptr.append(indptr[-1] + scores.shape[0])


In [3]:
scores = np.concatenate([np.load(f'../processed_data/AmpC_all{i}.npy') for i in range(10)])

# functions to handle the slabs

In [4]:
def extractFPs(chunkID, indptr, isTrain):
    fp = sparse.load_npz(f'../processed_data/AmpC_all{chunkID}.npz')
    mask = isTrain[indptr[chunkID]:indptr[chunkID+1]]
    return fp[mask]

def buildTrain(indptr, isTrain, verbose=0):
    if verbose:
        print('building training matrix')
    fps = sparse.vstack([extractFPs(i, indptr, isTrain) for i in range(10)])
    return fps

def chunkPredictProba(model, indptr, isTrain, verbose=0):
    if verbose:
        print('predicting probabilities')
    probas = []
    for chunkID in range(10):
        fps = extractFPs(chunkID, indptr, ~isTrain)
        proba = model.predict_proba(fps)[:,1]
        probas.append(proba)
    return np.concatenate(probas)

def chunkPredict(model, indptr, isTrain, verbose=0):
    if verbose:
        print('predicting probabilities')
    preds = []
    for chunkID in range(10):
        fps = extractFPs(chunkID, indptr, ~isTrain)
        pred = -1*model.predict(fps) #best scoring will now be on top (like the proba)
        preds.append(pred)
    return np.concatenate(preds)

# Train and RF regressor and Logistic Regression models

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
#model = LogisticRegression(max_iter=10000, C=0.1)
model = LogisticRegression(max_iter=10000, C=1)
#model = Ridge()

In [6]:
# #trainingSetSizes = np.array([0.4,0.2,0.1])/100
# trainingSetSizes = np.array([0.1])/100

# trainingSetSizes = (trainingSetSizes * scores.shape[0]).astype(int)

In [7]:
#topK = (scores.argsort().argsort() < (scores.shape[0]*0.005)) #0.5th percentile.

topK = (scores.argsort().argsort() < 50_000) #~0.05th percentile.
tot = topK.sum() 

# With Altair, using three repeats:

In [8]:

trainingSetSizes=[400_000, 200_000, 100_000]


for percentile in [0.55 ]:    
    df = pd.DataFrame(columns=['Algorithm', 'Training size', 'N ligands explored', '% top-k found'])
    count=0
    
    for i in range(3):
        idx = np.arange(scores.shape[0])
        np.random.shuffle(idx)

        for size in trainingSetSizes:
            #split indices into train and test:
            train = idx[:size].copy()
            test = idx[size:].copy()
            train.sort()
            test.sort()
    
            #generate a 'is a training instance' mask. 
            isTrain = np.zeros(scores.shape[0]).astype(bool)
            isTrain[train]=True
    
            #topK molecules already found in the training set:
            numFound = topK[train].sum()
        
            df.loc[count] = ['morgan_feat', size, train.shape[0], numFound/tot]
            count+=1
            print(count, numFound)
    
            #estimate the cutoff once, from the initial random sample:
            cutoff = np.percentile(scores[train], percentile)
            
            for i in range(5):

                #fit logreg model:
                model.fit(buildTrain(indptr, isTrain, 1), scores[isTrain]<cutoff)
                #fit ridge:
                #model.fit(buildTrain(indptr, isTrain, 1), scores[isTrain])
    
                #predict (slowest step) for logreg:
                proba = chunkPredictProba(model, indptr, isTrain, 1)
                #predict (slowest step) for ridge:
                #proba = chunkPredict(model, indptr, isTrain, 1)
    
                #rank the probabilities
                proba_sorted = (-proba).argsort()
        
                #rank the unseen instances:
                test = test[proba_sorted]

                #now append the next N instances from the rank ordered unseen instances onto the training set:
                train = np.concatenate([train, test[:size]])
        
                #update the isTrain mask:
                isTrain[train]=True
        
                #now remove those training instances from the test set:
                test = test[size:]

                #keep the train and test idx arrays sorted so they agree with the chunked* methods:
                test.sort()
                train.sort()
        
                #topK molecules already found in the training set:
                numFound = topK[train].sum()
            
                df.loc[count] = ['morgan_feat', size, train.shape[0], numFound/tot]
                count+=1
                print(count, numFound)
                #df.to_csv('../processed_data/ampc_reconstruction_'+str(percentile)+'_1_.csv')
                
    #df.to_csv('../processed_data/ampc_reconstruction_'+str(percentile)+'_1_.csv')

    

1 192
building training matrix
predicting probabilities
2 18449
building training matrix


KeyboardInterrupt: 

# Plot

In [32]:
pd.read_csv('../processed_data/ampc_reconstruction_0.15.csv')

,Unnamed: 0,Algorithm,Training size,N ligands explored,% top-k found
0,0,morgan_feat,400000,400000,0.00428
1,1,morgan_feat,400000,800000,0.37858
2,2,morgan_feat,400000,1200000,0.67542
3,3,morgan_feat,400000,1600000,0.80326
4,4,morgan_feat,400000,2000000,0.87552
5,5,morgan_feat,400000,2400000,0.91186
6,6,morgan_feat,200000,200000,0.00220
7,7,morgan_feat,200000,400000,0.19212
8,8,morgan_feat,200000,600000,0.43718
9,9,morgan_feat,200000,800000,0.57908


In [9]:
import altair as alt

In [12]:
prev_results = [['RF (Coley)', 400_000, 71.4, 2.1], ['NN (Coley)', 400_000, 74.7, 1.4],
                ['MPN (Coley)',400_000, 87.9, 2.3],
    ['RF (Coley)', 200_000, 45.5, 1.8],
['NN (Coley)', 200_000, 52.8, 0.5],
['MPN (Coley)', 200_000, 67.1, 2.1],
['RF (Coley)', 100_000, 24.0, 2.2],
['NN (Coley)', 100_000 , 33.3,0.3],
['MPN (Coley)', 100_000, 52.0, 0.5]]


In [13]:


coley = pd.DataFrame(columns=['Algorithm', 'Training size', 'N ligands explored', '% top-k found'])
count = 0 
for res in prev_results:
    
    desired_std_dev = res[3]
    samples = np.array([-1,0,1]).astype(float)
    samples *= (desired_std_dev/np.std(samples))
    for s in samples:
        coley.loc[count]= [res[0], res[1], res[1]*6, (s+res[2])/100]
        count+=1

In [14]:
concat = pd.concat([df, coley])

In [15]:
error_bars = alt.Chart(concat).mark_errorbar(extent='ci').encode(
  x=alt.X('N ligands explored:Q',title='Number of ligands sampled'),
  y=alt.Y('% top-k found:Q', title='% top 50,000 found'),
    color=alt.Color('Algorithm')
)

points = alt.Chart(concat).mark_point(filled=True, color='black').encode(
  x=alt.X('N ligands explored:Q'),
  y=alt.Y('% top-k found:Q',aggregate='mean',title='% top 50,000 found'),
    color=alt.Color('Algorithm')
)

line = alt.Chart(concat).mark_line(color='black',size=1,opacity=0.5).encode(
  x=alt.X('N ligands explored:Q'),
  y=alt.Y('% top-k found:Q',aggregate='mean',title='% top 50,000 found'),
    color=alt.Color('Algorithm')
)

ch = (error_bars+points+line).properties(height=400,width=200).facet(
    column=alt.Column('Training size:N',sort=alt.Sort([0.004, 0.002, 0.001])),
).resolve_scale(x='independent')
ch

alt.FacetChart(...)

In [33]:
chs = []
for frac in [400000, 200000, 100000]:
    
    df_ = concat[concat['Training size']==frac].replace('morgan_feat', 'Morgan pharm. & Log.reg. (ours)')
    error_bars = alt.Chart(df_).mark_errorbar(extent='ci').encode(
          x=alt.X('N ligands explored:Q', 
                  title='Number of ligands sampled',
                  scale=alt.Scale(domain=[0,max(df_['N ligands explored'])+10000])),
          y=alt.Y('% top-k found:Q',scale=alt.Scale(domain=[0,0.95])),
        color=alt.Color('Algorithm')
        )

    points = alt.Chart(df_).mark_point(filled=True, color='black').encode(
          x=alt.X('N ligands explored:Q'),
          y=alt.Y('% top-k found:Q',aggregate='mean',scale=alt.Scale(domain=[0,0.95])),
        color=alt.Color('Algorithm')
        )

    line = alt.Chart(df_).mark_line(color='black',size=1,opacity=0.5).encode(
          x=alt.X('N ligands explored:Q'),
          y=alt.Y('% top-k found:Q',aggregate='mean',scale=alt.Scale(domain=[0,0.95])),
        color=alt.Color('Algorithm')
        )
    ch = (error_bars+points+line).properties(width=200)
    ch.title = str(frac / (100*1e6)*100)
    chs.append(ch)


In [34]:
chs[0]
sup = chs[0] |  chs[1] | chs[2]
#sup.save('../figures/ampC_reconstruction.html') #using 0.05

In [35]:
chs[0]+chs[1]+chs[2]

alt.LayerChart(...)

In [289]:
sup.save('../figures/ampC_reconstruction.html')